# Project Title: Examine the Key Differences in Funding Patterns Before and After Major Economic Events

## Introduction 

By using "startup investments" dataset, the research is about how local government policies for economic conditions is different regions influence the amount and frequency of startup investments.

## economic environment

In [1]:
pip install -q nbconvert


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -q pyppeteer


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -q qeds


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

from IPython.display import display

In [8]:
objects = pd.read_csv("/Users/zzybollar/Downloads/archive/objects.csv")

/var/folders/lh/v36j2zlj0t9_xqpxcj3l9qfw0000gn/T/ipykernel_9161/3389580221.py:1: DtypeWarning: Columns (3,7,9,10,17,18,21,22,23,25,26,29,30,37) have mixed types. Specify dtype option on import or set low_memory=False.
  objects = pd.read_csv("/Users/zzybollar/Downloads/archive/objects.csv")


In [9]:
offices = pd.read_csv("/Users/zzybollar/Downloads/archive/offices.csv")
unique_countries = offices["country_code"].unique()
unique_countries

array(['USA', 'ISR', 'GBR', 'LUX', 'CAN', 'RUS', 'DEU', 'IRL', 'FRA',
       'FIN', 'NLD', 'CHE', 'AFG', 'KOR', 'ITA', 'JPN', 'SGP', 'AUS',
       'SWE', 'BRA', 'ESP', 'DNK', 'CHN', 'ZAF', 'IDN', 'IND', 'NZL',
       'BEL', 'DOM', 'CZE', 'POL', 'THA', 'ARG', 'EST', 'GIN', 'CYP',
       'PRT', 'HKG', 'ROM', 'EGY', 'QAT', 'AUT', 'NOR', 'MTQ', 'BGR',
       'ARE', 'TUR', 'VNM', 'MUS', 'LVA', 'CHL', 'UKR', 'SVN', 'HUN',
       'NIC', 'BRB', 'TUN', 'UMI', 'SAU', 'PHL', 'CRI', 'GRC', 'VGB',
       'PAK', 'ISL', 'LTU', 'MEX', 'PER', 'NGA', 'GIB', 'CYM', 'PRK',
       'TWN', 'MYS', 'ANT', 'MLT', 'URY', 'BIH', 'HRV', 'JOR', 'BGD',
       'KEN', 'UZB', 'MKD', 'ARM', 'CIV', 'SDN', 'MAR', 'BOL', 'CSS',
       'LBN', 'AZE', 'LKA', 'KGZ', 'VEN', 'RWA', 'UGA', 'BHR', 'KWT',
       'WSM', 'BLR', 'COL', 'REU', 'ARA', 'PRI', 'ATG', 'ECU', 'PAN',
       'KAZ', 'LIE', 'LAO', 'ALB', 'BMU', 'GHA', 'IRQ', 'BHS', 'JAM',
       'OMN', 'CMR', 'CUB', 'IRN', 'BLZ', 'MMR', 'SWZ', 'FST', 'SVK',
       'AGO', 'MDA',

In [10]:
US_offices = offices[offices["country_code"]=="USA"]
US_offices = US_offices[["id","region","city","zip_code","state_code","country_code"]]

In [44]:
life_expectancy = pd.read_csv("datasets/life expectancy.csv")
life_expectancy.rename(columns = {"Country Code": "country_code"},inplace = True)
combined_data_1 = pd.merge(offices, life_expectancy, on = 'country_code',how='inner')
combined_data_1 = combined_data_1[["id","region","city","zip_code",
                                   "country_code","Life Expectancy World Bank"]]
combined_data_1.drop_duplicates(inplace=True)
combined_data_1


,id,region,city,zip_code,country_code,Life Expectancy World Bank
0,1,Seattle,Seattle,98104,USA,76.836585
1,1,Seattle,Seattle,98104,USA,76.936585
2,1,Seattle,Seattle,98104,USA,77.036585
3,1,Seattle,Seattle,98104,USA,77.487805
5,1,Seattle,Seattle,98104,USA,77.687805
...,...,...,...,...,...,...
2084997,112718,SF Bay,San Mateo,94403,USA,78.841463
2084998,112718,SF Bay,San Mateo,94403,USA,78.690244
2084999,112718,SF Bay,San Mateo,94403,USA,78.539024
2085001,112718,SF Bay,San Mateo,94403,USA,78.639024


In [10]:
investments = pd.read_csv("/Users/zzybollar/Downloads/archive/investments.csv")
investments = investments.drop(columns=['id', 'created_at', 'updated_at'])
investments.drop_duplicates()

,funding_round_id,funded_object_id,investor_object_id
0,1,c:4,f:1
1,1,c:4,f:2
2,3,c:5,f:4
3,4,c:5,f:1
4,4,c:5,f:5
...,...,...,...
80897,57882,c:285244,p:267312
80898,57912,c:60389,p:268570
80899,57929,c:286183,p:365
80900,57929,c:286183,p:176466
